In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import mysql.connector as sql
 
app = dash.Dash()
db_connection = sql.connect(host='127.0.0.1',
                                         database='python',
                                         user='root',
                                         password='Nihal@123'
                                        )

db_cursor = db_connection.cursor()

db_cursor.execute('SELECT * FROM xn')

table_rows = db_cursor.fetchall()
df = pd.DataFrame(table_rows)
df.rename(columns={0: "Date", 1: "Website_action", 2:"Media",3:"FrequencyOfVoice", 4:"Potential_Customers", 5:"Articles_per_month",
                  6:"Positive_reviews",7:"Negative_reviews",8:"Graphnames"}, inplace = True)

#df['Date']= pd.to_datetime(df['Date'])
#df.drop(columns = ['GraphNames'], inplace=True)
file = df
db_cursor.execute('SELECT * FROM Graph')

graph_names = db_cursor.fetchall()
graph_names = pd.DataFrame(graph_names).rename(columns={0:"GraphNames"})
db_connection.close()
"""

df = pd.read_excel('E:\\Chitra\\NU\\XN-Fall2019\\XN_Draft_final.xlsx')
file = pd.read_excel("C:\\Users\\bhara\\Downloads\\XN_Draft_final.xlsx")
"""
file['Year'],file['Month'],file['day']=file.Date.dt.year,file.Date.dt.month,file.Date.dt.day
file.head()
file['Year'],file['Month'],file['day']=file.Year.astype(int),file.Month.astype(int),file.day.astype(int)

file['week'] = file.Date.dt.week
potential_customers_2016 = file[file['Year']==2016]
pie_file = potential_customers_2016.groupby(['Media']).sum().reset_index()

def setyear(row):
    if row == 1:
        return "Jan"
    if row == 2:
        return "Feb"
    if row == 3:
        return "Mar"
    if row == 4:
        return "Apr"
    if row == 5:
        return "May"
    if row == 6:
        return "Jun"
    if row == 7:
        return "Jul"
    if row == 8:
        return "Aug"
    if row == 9:
        return "Sep"
    if row == 10:
        return "Oct"
    if row == 11:
        return "Nov"
    if row == 12:
        return "Dec"

pie_file = potential_customers_2016.groupby(['Media']).sum().reset_index()
features = df.columns
dropdown_names = graph_names['GraphNames']

    
article_file = potential_customers_2016.groupby(['Month']).sum().reset_index()
article_file.head()
article_file['Month'] = article_file['Month'].apply(lambda row: setyear(row))


app.layout = html.Div([
    html.Div([
        html.Div([
            html.H4(children='Choose graph names from dropdown'),
            dcc.Dropdown(
                id='xaxis',
                options=[{'label': i, 'value': i} for i in dropdown_names],
                value='CustomerVisit'
            )
        ],
        style={'width': '48%', 'display': 'inline-block'})
    ]),
 
    dcc.Graph(id='feature-graphic'),
    html.Div(id='display-selected-values')
    
], style={'padding':10})

@app.callback(
    Output('feature-graphic', 'figure'),
    [Input('xaxis', 'value')])
def update_graph(xaxis_name):
    
    if xaxis_name == "Articles":
        trace_a = go.Bar(x=article_file['Month'],
                         y=article_file['Articles_per_month'],
                         name="Articles per month",
                         marker=dict(color='#A2D5F2'))
        return {'data': go.Data([trace_a]),'layout': go.Layout(
                title="Number Of Articles",
                xaxis={'title': "Month"},
                yaxis={'title': "Articles"})}
        
    elif xaxis_name == "Reviews":
        
        trace_a = go.Bar(x=article_file['Month'],
                         y=article_file['Positive_reviews'],
                         name="Positive reviews",
                         marker=dict(color='#A2D5F2'))
        trace_b = go.Bar(x=article_file['Month'],
                         y=article_file['Negative_reviews'],
                         name="Negative reviews",
                         marker=dict(color='#FFCDD2'),
                         text = "sdgsd")
        return {        
            'data': go.Data([trace_a,trace_b]),
            'layout': go.Layout(
                title="Positive and Negative Reviews",
                xaxis={'title': "Month"},
                yaxis={'title': "Review"}
                
            )}
        
    
    elif xaxis_name == "PowerOfVoice":
               
        trace_a1 = go.Pie(labels=pie_file['Media'].unique().tolist(),
                         values=pie_file['FrequencyOfVoice'],hole=.3,
                         name="Pie chart")
        return {
            
        
            'data': go.Data([trace_a1]),
            "layout": go.Layout(title="Power of Voice in Different Media")
            
        }
    elif xaxis_name == "CustomerVisit":
        potential_customers_2016 = file[file['Year']==2016]
        df1 = potential_customers_2016.groupby(['week', 'Potential_Customers'], as_index=False).sum()
        df1 = df1.groupby('week')['Potential_Customers'].sum().reset_index()
        plot_trace1 = go.Scatter(x=df1['week'],
                         y =df1['Potential_Customers'],
                         name= "Potential Customers By Week",
                         mode='lines+markers')

        #plot_data = go.Data([plot_trace1])
        
        return {
             'data': go.Data([plot_trace1]),
            "layout": go.Layout(title="Customer Visited Website per week",
                               xaxis={'title': "Week Number"},
                yaxis={'title': "Number Of Visitors"})
            
        }
    elif xaxis_name =="VisitorsByState":
        df2 = pd.read_excel('C:\\Users\\bhara\\Downloads\\XN_Draft.xlsx','Sheet2')
        trace_a = go.Line(x=df2['Months'],
                y=df2['NYC'],
                name="NYC",
                marker=dict(color='#ffa500'))

        trace_b = go.Line(x=df2['Months'],
                y=df2['LA'],
                name="LA",
                marker=dict(color='#0000ff'),
                )
        trace_c = go.Line(x=df2['Months'],
                y=df2['Seattle'],
                name="Seattle",
                marker=dict(color='#00ff00'),
               )
        return {
            'data': go.Data([trace_a,trace_b,trace_c]),
            'layout': go.Layout(
                title  = "Number of Visitors by States",
                xaxis={'title': "Months"},
                yaxis={'title': "Number of Visitors"})
        }
    elif xaxis_name =="WebsiteAction":
        trace_d = go.Line(x=file['Date'],
                          y=file['Website_action'],
                          name="NYC",
                          marker=dict(color='#6a0dad'))

        return {
            'data': go.Data([trace_d]),
            'layout': go.Layout(
                title  = "Onclusive website action",
                xaxis={'title': "Month"},
                yaxis={'title': "Clicks"})
        }
        
    else:
        trace_a = go.Bar(x=article_file['Month'],
                         y=article_file['Articles_per_month'],
                         name="Articles per month",
                         marker=dict(color='#A2D5F2'))
        return {
        
            'data': go.Data([trace_a]),
            'layout': go.Layout(
                xaxis={'title': "sjfghsdhjsdk"},
                yaxis={'title': "Articles"}
                
            )
        }
@app.callback(
    Output('display-selected-values', 'children'),
    #Output('my-text', component_property='children'),
    [Input('feature-graphic', 'figure')])
    
def send_print(figure):
        if figure['layout']['title']['text'] =="Onclusive website action":
            return u'this hrapojfse is forn onslcuwns website'
        elif figure['layout']['title']['text'] =="Power of Voice in Different Media":
            return u'This pie chart shows various media outlets and their market share within the United States. In relation to Onclusive, a pie chart such as this can prove to be very important for investors, the media outlets themselves, and the public. For example, an investor might want to know where is the best investment. A CEO might want to know their market share. Lastly, the public might be interested to know if one firm is monopolizing the others.In the US, it is very important to be able to express free speech. If one firm has established a monopoly over all the others then free speech is violated as they will be in control of the majority of media publications coming out to the public. Thus, this pie chart can clearly show the presence of all the different media publications.'
        elif figure['layout']['title']['text'] =="Number Of Articles":
            return u'The below bar plot shows the number of articles posted by each month in different media. When compared to other months, there are a greater number of articles posted in April, October, June, July months.  There are almost 170 articles posted in April and October month. There are very few articles 1around 20 posted in February. Based on this We can also calculate the Potential customers for each month by calculating how many people are visiting websites after seeing the article and which media posting a greater number of articles. This analysis will be useful for recommending more articles to users in order to improve website clicks thereby profits.'
        elif figure['layout']['title']['text'] =="Positive and Negative Reviews":
            return u'The reviews for the past year analyzed. Positive and Negative reviews from various media channels were collected.The results are plotted.During the start of PR analytics adoption(Jan to Jul) there were high amount of Negative feedback about the company almost twice the number of positive reviews.Through PR analytics adoption,the company got a better insight into how the customers felt about the brand. Most common issue mentioned in the negative feedback was related to customer service which was addressed and the change in customer service team resulted in a drop in negative reviews begining from August.Currently negative reviews are lower compared to positive reviews and the company should continue monitoring the customer feedbacks to address critical issues.'
        elif figure['layout']['title']['text'] =="Number of Visitors by States":
            return u'The below analysis shows monthly trend about number of vistors visited by states and also tells which month has maximum number of visitors in each state. This information tells us that during month of september maximum number of visitors visited in each state and out of all three states visitors coming from LA was maximum.This information can help PR team to know their audience based on demographics,their interests and what proportion of visitors are new compared with those who visited before.It will also help in doing better planning for next makreting campaign and will give idea who will your target audience.'
        elif figure['layout']['title']['text'] =="Customer Visited Website per week":
            return u'Top 10 weeks with maximum number of potential users week 3.This information tells us that during these ten weeks maximum people visited the brand’s website after reading the article associated with the brand. These people can be considered as potential customers as they have shown interest in the brand and visited the website. Based on this information, PR team can identify which articles or blogs they posted during these ten weeks, that persuaded more people to visit their brand’s website and they can create similar content in the future'
        else:
            return u'n grkjshwe text'
   
    
if __name__ == '__main__':
    app.run_server()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Feb/2020 19:40:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2020 19:40:02] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2020 19:40:02] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2020 19:40:02] "GET /_dash-component-suites/dash_core_components/async~plotlyjs.v1_5_1m1573761749.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2020 19:40:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2020 19:40:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2020 21:36:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Feb/2020 21:36:55] "POST /_dash-update-component HTTP/1.1" 200 -


In [ ]:
article_file